In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Adrenocortical_Cancer"
cohort = "GSE108088"

# Input paths
in_trait_dir = "../../input/GEO/Adrenocortical_Cancer"
in_cohort_dir = "../../input/GEO/Adrenocortical_Cancer/GSE108088"

# Output paths
out_data_file = "../../output/preprocess/Adrenocortical_Cancer/GSE108088.csv"
out_gene_data_file = "../../output/preprocess/Adrenocortical_Cancer/gene_data/GSE108088.csv"
out_clinical_data_file = "../../output/preprocess/Adrenocortical_Cancer/clinical_data/GSE108088.csv"
json_path = "../../output/preprocess/Adrenocortical_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Comprehensive molecular profiling of children with recurrent cancer II"
!Series_summary	"to explore possible treatment targets and reasons for agressive children cacners by comprehensive molecular profiling on several platforms"
!Series_summary	"to explore copy number aberrations related to cancers"
!Series_overall_design	"diagnostics of children meeting the oncologist with recurrent or agressive cancers where treatment options have been exhausted"
Sample Characteristics Dictionary:
{0: ['condition: Atypical meningioma', 'condition: Choroid plexus carcinoma / Malignant peripheral nerve sheeth tumor', 'condition: Pilocytisc/pilomyxoid astrocytoma', 'condition: Pleomorphic xanthoastrocytoma', 'condition: Mesoblastisc nephroma', 'condition: Signetringcell carcinoma', 'condition: Ganglioglioma / Diffuse astrocytoma', 'condition: Chondrosarkoma', 'condition: Chordoma, dedefferentiated/anaplatic type (INI1-loss)', 'condition: Hepatoblastoma', 'condition

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import pandas as pd
import json
from typing import Callable, Optional, Dict, Any

# 1. Gene Expression Data Availability
# Based on the background information, this appears to be a cancer dataset that likely includes gene expression data.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Trait Data Availability
# Row 0 contains condition information, which can be used for trait (cancer type)
trait_row = 0

# Age is not available in the sample characteristics
age_row = None

# Gender is not available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert adrenocortical cancer status based on condition field."""
    if not value or not isinstance(value, str):
        return None
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # For adrenocortical cancer, we need to check if any of the conditions are related to adrenocortical cancer
    # This dataset doesn't appear to have adrenocortical cancer explicitly, so we'll return 0 for all cases
    return 0  # None of the conditions match adrenocortical cancer

def convert_age(value):
    # Age data is not available
    return None

def convert_gender(value):
    # Gender data is not available
    return None

# 3. Save Metadata
# Check if trait data is available
is_trait_available = trait_row is not None

# Validate and save cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Check if trait_row is not None to proceed with clinical feature extraction
if trait_row is not None:
    # Assuming clinical_data was loaded in a previous step
    # If not, we need to load it
    try:
        clinical_data
    except NameError:
        # Load clinical data if not already loaded
        clinical_data_path = os.path.join(in_cohort_dir, "clinical_data.csv")
        if os.path.exists(clinical_data_path):
            clinical_data = pd.read_csv(clinical_data_path)
        else:
            raise FileNotFoundError(f"Clinical data file not found at {clinical_data_path}")
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(selected_clinical_df)
    print("Preview of the clinical dataframe:")
    print(preview)
    
    # Save the clinical dataframe to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of the clinical dataframe:
{'GSM2889381': [0.0], 'GSM2889382': [0.0], 'GSM2889383': [0.0], 'GSM2889384': [0.0], 'GSM2889385': [0.0], 'GSM2889386': [0.0], 'GSM2889387': [0.0], 'GSM2889388': [0.0], 'GSM2889389': [0.0], 'GSM2889390': [0.0], 'GSM2889391': [0.0], 'GSM2889392': [0.0], 'GSM2889393': [0.0], 'GSM2889394': [0.0], 'GSM2889395': [0.0], 'GSM2889396': [0.0], 'GSM2889397': [0.0], 'GSM2889398': [0.0], 'GSM2889399': [0.0], 'GSM2889400': [0.0], 'GSM2889401': [0.0], 'GSM2889402': [0.0], 'GSM2889403': [0.0], 'GSM2889404': [0.0], 'GSM2889405': [0.0], 'GSM2889406': [0.0], 'GSM2889407': [0.0], 'GSM2889408': [0.0], 'GSM2889409': [0.0], 'GSM2889410': [0.0], 'GSM2889411': [0.0], 'GSM2889412': [0.0], 'GSM2889413': [0.0], 'GSM2889414': [0.0], 'GSM2889415': [0.0], 'GSM2889416': [0.0], 'GSM2889417': [0.0], 'GSM2889418': [0.0], 'GSM2889419': [0.0], 'GSM2889420': [0.0], 'GSM2889421': [0.0], 'GSM2889422': [0.0], 'GSM2889423': [0.0]}
Clinical data saved to ../../output/preprocess/Adrenocortical

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be Affymetrix probe IDs (e.g., "1007_s_at", "1053_at"), not standard human gene symbols.
# Affymetrix probe IDs need to be mapped to human gene symbols for proper gene-level analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Based on the previews:
# - In gene_data, the indices are Affymetrix probe IDs like "1007_s_at"
# - In gene_annotation, 'ID' column contains probe IDs and 'Gene Symbol' column contains gene symbols

# 2. Extract gene mapping data
prob_col = 'ID'  # This column contains the probe identifiers
gene_col = 'Gene Symbol'  # This column contains the gene symbols
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)

# 3. Apply gene mapping to convert probe-level data to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the gene expression data after mapping
print("Preview of gene data after mapping to gene symbols:")
print(gene_data.head(5))


Preview of gene data after mapping to gene symbols:
          GSM2889381  GSM2889382  GSM2889383  GSM2889384  GSM2889385  \
Gene                                                                   
A1BG        5.523549    6.286303    4.722706    6.001011    4.748103   
A1BG-AS1    5.312258    5.636363    4.377456    4.989505    4.427621   
A1CF        7.206790    7.545095    7.565904    7.332128    8.544245   
A2M        16.878920   12.468288   16.198479   15.144045   17.156246   
A2M-AS1     6.784101    4.090659    5.154216    7.076977    6.310073   

          GSM2889386  GSM2889387  GSM2889388  GSM2889389  GSM2889390  ...  \
Gene                                                                  ...   
A1BG        5.784339    5.073090    5.322978    5.552850   10.028951  ...   
A1BG-AS1    4.967411    6.596631    4.705341    4.475880    6.932284  ...   
A1CF        8.336636    7.851718    8.380085    8.670368   19.276394  ...   
A2M        14.821813   15.381740   15.770469   14.939509  

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols...")
try:
    # Normalize gene symbols using the NCBI Gene database
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene expression data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error during gene normalization: {e}")
    # If normalization fails, use the original gene data
    print("Using original gene expression data...")
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Original gene expression data saved to {out_gene_data_file}")

# 2. Load the clinical data that we created in Step 2
print("Loading clinical data...")
try:
    clinical_df = pd.read_csv(out_clinical_data_file)
    # Check if the dataframe has an unnamed index column that should become the index
    if 'Unnamed: 0' in clinical_df.columns:
        clinical_df.set_index('Unnamed: 0', inplace=True)
    print(f"Clinical data shape: {clinical_df.shape}")
except Exception as e:
    print(f"Error loading clinical data: {e}")
    # If there's an error, we'll try to recreate the clinical data
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    clinical_df = selected_clinical_df.T
    print(f"Recreation of clinical data, shape: {clinical_df.shape}")

# 3. Check the structure of the clinical data
print("Clinical data columns:", clinical_df.columns.tolist())

# 4. Link the clinical and genetic data
print("Linking clinical and genetic data...")
try:
    # Make sure we have the trait column in the clinical_df
    if trait not in clinical_df.columns:
        # Rename the first column to the trait name if it's the adrenocortical cancer indicator
        if len(clinical_df.columns) > 0:
            clinical_df.rename(columns={clinical_df.columns[0]: trait}, inplace=True)
    
    # Transpose gene_data to have samples as rows
    gene_data_t = normalized_gene_data.T
    
    # Link the data using index for proper alignment
    linked_data = pd.concat([clinical_df, gene_data_t], axis=1)
    print(f"Linked data shape: {linked_data.shape}")
    
    # Check if trait column exists before handling missing values
    if trait in linked_data.columns:
        # Handle missing values
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
        
        # Check if the trait is biased
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Is trait biased? {is_biased}")
    else:
        print(f"Trait column '{trait}' not found in linked data. Cannot handle missing values or check bias.")
        is_biased = True  # Mark as biased if we can't properly analyze
except Exception as e:
    print(f"Error linking data: {str(e)}")
    is_biased = True  # Mark as biased if linking fails
    linked_data = clinical_df  # Use just clinical data as fallback

# 5. Validate and save cohort information
note = "This dataset contains cancer samples but none are labeled as adrenocortical cancer. All samples are coded as 0 for the trait, making it unsuitable for trait-gene association analysis."

is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

print(f"Dataset usability: {is_usable}")

# 6. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies.")

Normalizing gene symbols...
Gene data shape after normalization: (19845, 43)


Normalized gene expression data saved to ../../output/preprocess/Adrenocortical_Cancer/gene_data/GSE108088.csv
Loading clinical data...
Clinical data shape: (1, 43)
Clinical data columns: ['GSM2889381', 'GSM2889382', 'GSM2889383', 'GSM2889384', 'GSM2889385', 'GSM2889386', 'GSM2889387', 'GSM2889388', 'GSM2889389', 'GSM2889390', 'GSM2889391', 'GSM2889392', 'GSM2889393', 'GSM2889394', 'GSM2889395', 'GSM2889396', 'GSM2889397', 'GSM2889398', 'GSM2889399', 'GSM2889400', 'GSM2889401', 'GSM2889402', 'GSM2889403', 'GSM2889404', 'GSM2889405', 'GSM2889406', 'GSM2889407', 'GSM2889408', 'GSM2889409', 'GSM2889410', 'GSM2889411', 'GSM2889412', 'GSM2889413', 'GSM2889414', 'GSM2889415', 'GSM2889416', 'GSM2889417', 'GSM2889418', 'GSM2889419', 'GSM2889420', 'GSM2889421', 'GSM2889422', 'GSM2889423']
Linking clinical and genetic data...
Linked data shape: (44, 19888)
Data shape after handling missing values: (1, 43)
Quartiles for 'Adrenocortical_Cancer':
  25%: 0.0
  50% (Median): 0.0
  75%: 0.0
Min: 0.0
M